# Mini-projet : Classification de tumeurs gliales

Les _gliomes_ ou _tumeurs gliales_ sont des tumeurs du glie, le tissu de soutien neuronal du cerveau. Elles sont classifiées en 4 grades anatomo-pathologiques, dont dépend la prise en charge.

Dans ce jeu de données, chaque observation est un gliome, décrit par l'expression de 4 434 gènes. L'expression d'un gène est une mesure de la quantité d'ARN correspondant à ce gène qui est présente dans la cellule. Schématiquement, l'ADN est transcrit en ARN, lequel est lui-même traduit en une protéine. Les protéines assurent une multitude de fonctions du vivant, mais mesurer leur quantité est difficile ; d'où l'intérêt d'utiliser les quantités d'ARN, bien que la correspondance ne soit pas immédiate. 

Chaque gliome de notre jeu de données est étiquetée en fonction de son grade. 

Le but de ce projet est de construire un classifieur qui détermine, sur la base de l'expression de ces 4 434 gènes, le grade d'un gliome.

## Instructions
1. Comparez les performances d'au moins deux algorithmes d'apprentissage sur ce problème de classification.

__Attention :__
* au _data leakage_ (ne pas utiliser les données sur lesquelles on évalue les modèles pour les entraîner ou prétraiter les données)
* à la taille du jeu de données
* au nombre de classes
* à choisir une mesure de performance appropriée (justifiez votre choix)
2. Identifiez, quand cela est possible, les gènes les plus importants pour les modèles que vous avez entraînés. S'agit-il des mêmes gènes
* entre deux modèles obtenus grâce à un algorithme d'apprentissage différents ?
* entre deux modèles obtenus en utilisant le même algorithme d'apprentissage sur des sous-échantillons différents des données ?

N'oubliez pas de commenter et interpréter vos résultats.

## Chargement des données

In [44]:
import scipy.io
data_matrix = scipy.io.loadmat("gliome.mat")

In [45]:
X = data_matrix['X']
print(X.shape)

(50, 4434)


In [46]:
y = data_matrix['Y'][:, 0] 
print(y.shape)
random_state = 42

(50,)


In [47]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, GridSearchCV

X_std = StandardScaler().fit_transform(X)
# pca = PCA(n_components=0.95)
# pca.fit(X_std)
# X_std = pca.transform(X_std)

X_train, X_test, y_train, y_test = train_test_split(X_std, y, test_size=0.2, random_state=random_state)

* Obs, j'ai pas des connaissances de médicine mais les idées ici presentées peut-être modifiés facilement en aient un spécialiste en médecine.
On va utiliser la metric "accuracy" parce que nos données sont bien stratifiés, c'est-à-dire, on a la même quantidate de classes pour chaque classe y_i.
Cependant, on peut penser que la précision de la classe 4 est plus importante que la précision de la classe 1, parce que la classe 4 démande un chirurgie et on veut pas faire des opérations sans être nécessaire. D'autre, le recall de classe 1 est très importante aussi, parce que si on découvre le gliome à sont débout alors on peut le traiter et ênpecher son agravément.

In [52]:
from collections import defaultdict
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_val_predict, StratifiedKFold
from math import sqrt

models = []
models.append(('LR', LogisticRegression(max_iter=1000,solver="saga"))) # saga = solver avec l1 regularization, ce qui est bien quand on a n << p. Elastic net serait le mieux à ce cas là probablement
models.append(('SVM', SVC(gamma='auto')))
models.append(('RF', RandomForestClassifier(n_estimators=100, max_depth=5, random_state=random_state,max_features='sqrt')))
models.append(('MLP', MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000)))

results = {}
names = []
n_classes = 4
for name, model in models:
    # Manual Stratified K-Fold pour avoir plus de controle et tous les metriques
    accuracy_list = []
    class_results = defaultdict(lambda: defaultdict(list))
    kf = StratifiedKFold(n_splits=5, random_state=random_state, shuffle=True).split(X_train, y_train)
    for train_index, test_index in kf:

        X_train_fold, X_test_fold = X_train[train_index], X_train[test_index]
        y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]

        model.fit(X_train_fold, y_train_fold)
        predict = model.predict(X_test_fold)
        # Fonction facile pour faire la precision et recall (et f1) de chaque class
        class_report = classification_report(y_test_fold, predict,zero_division=0,output_dict=True)
        for i in range(n_classes):
            class_results[i+1]['precision'].append(class_report[str(i+1)]['precision'])
            class_results[i+1]['recall'].append(class_report[str(i+1)]['recall'])
            class_results[i+1]['f1-score'].append(class_report[str(i+1)]['f1-score'])
            class_results[i+1]['support'].append(class_report[str(i+1)]['support'])
        accuracy_list.append(class_report['accuracy'])

    for i in range(1,n_classes+1):
        class_results[i]['precision'] = sum(class_results[i]['precision']) / len(class_results[i]['precision'])
        class_results[i]['recall'] = sum(class_results[i]['recall']) / len(class_results[i]['recall'])
        class_results[i]['f1-score'] = sum(class_results[i]['f1-score']) / len(class_results[i]['f1-score'])
        # support au cass où on veut weighted
    total_accuracy = sum(accuracy_list) / len(accuracy_list)
    std_accuracy = np.std(accuracy_list)

    results[model] = {"class_results":class_results, "accuracy":total_accuracy, "std_accuracy":std_accuracy}
    print('%s: %f (%f)' % (name, total_accuracy, sqrt(std_accuracy)))


LR: 0.775000 (0.407224)
SVM: 0.675000 (0.387298)
RF: 0.775000 (0.428616)
MLP: 0.600000 (0.428616)
